In [ ]:
import pandas as pd
import numpy as np

In [ ]:
devseen = pd.read_json('dev_seen.jsonl', lines=True).sort_values(by='id')
devseen.head()

,id,img,label,text
494,1268,img/01268.png,0,father and daughter bonding time is the best l...
159,1456,img/01456.png,1,they see them rollin..... they hating..
332,1726,img/01726.png,1,a real man loads the dishwasher every night
220,1742,img/01742.png,0,in just one hour from now i'll only have 4 hou...
217,1765,img/01765.png,1,"after a girl dies, what organ in her body stay..."


In [ ]:
baseline = pd.read_csv('visual_bert_coco.csv').sort_values(by='id')
baseline.head()

,id,proba,label
362,1268,0.000417,0
59,1456,0.100073,0
62,1726,0.837024,1
426,1768,0.000428,0
495,1796,0.269773,0


In [ ]:
vilio = pd.read_csv('FIN_dev_seen_ENS_SA_LOOP_2.csv')
vilmax = vilio['proba'].values.max()
vilmin = vilio['proba'].values.min()
vilio['proba'] = (vilio['proba'].values - vilmin) / (vilmax - vilmin)
vilio['label'] = np.where(vilio['proba'] > 0.5, 1, 0)

rename_columns = {'id_y': 'id', 'proba_x': 'proba', 'label_x': 'label'}
vilio = pd.merge(left=vilio, right=baseline, left_index=True, right_index=True, how='inner')
vilio = vilio.sort_values(by='id_y')[['id_y', 'proba_x', 'label_x']].rename(columns=rename_columns)
vilio

,id,proba,label
362,1268,0.659967,1
59,1456,0.768284,1
62,1726,0.909551,1
426,1768,0.652788,1
495,1796,0.160565,0
...,...,...,...
79,98235,0.013069,0
147,98516,0.222382,0
285,98543,0.716129,1
282,98701,0.093028,0


In [ ]:
rename_columns = {'label_x': 'truth', 'label_y': 'vilio_label', 'label': 'baseline_label', 'proba_x': 'vilio_proba', 'proba_y': 'baseline_proba'}
dev_vil = pd.merge(left=devseen, right=vilio, how='inner', left_on='id', right_on='id')
print(dev_vil.shape)
dev_vil_base = pd.merge(left=dev_vil, right=baseline, how='inner', left_on='id', right_on='id').rename(columns=rename_columns)
dev_vil_base

(372, 6)


,id,img,truth,text,vilio_proba,vilio_label,baseline_proba,baseline_label
0,1268,img/01268.png,0,father and daughter bonding time is the best l...,0.659967,1,0.000417,0
1,1456,img/01456.png,1,they see them rollin..... they hating..,0.768284,1,0.100073,0
2,1726,img/01726.png,1,a real man loads the dishwasher every night,0.909551,1,0.837024,1
3,1796,img/01796.png,1,life hack #23 how to get stoned with no weed,0.160565,0,0.269773,0
4,1925,img/01925.png,1,i am not racist i just don't like brown sugar,0.183132,0,0.000957,0
...,...,...,...,...,...,...,...,...
367,97453,img/97453.png,0,potato is my favorite color of the alphabet,0.138498,0,0.002182,0
368,98235,img/98235.png,0,fashion forward !,0.013069,0,0.017090,0
369,98543,img/98543.png,1,upper downer,0.716129,1,0.957242,1
370,98701,img/98701.png,1,oh no i can't steal anymore,0.093028,0,0.000820,0


In [ ]:
vil_true = dev_vil_base.loc[dev_vil_base['truth'] == dev_vil_base['vilio_label']]
vil_true[vil_true['truth'] == vil_true['baseline_label']]

,id,img,truth,text,vilio_proba,vilio_label,baseline_proba,baseline_label
2,1726,img/01726.png,1,a real man loads the dishwasher every night,0.909551,1,0.837024,1
5,1967,img/01967.png,0,when you decide to stop racism by burning its ...,0.215077,0,0.000817,0
10,2518,img/02518.png,0,when the dog bites you and you bite back to as...,0.172882,0,0.000374,0
12,2634,img/02634.png,1,and just like that... this sandwich maker doub...,0.714565,1,0.977146,1
14,3214,img/03214.png,0,bartender: how about a nice 12 yr old? him: bo...,0.219183,0,0.016573,0
...,...,...,...,...,...,...,...,...
360,96250,img/96250.png,0,i'm going home i'm russian,0.072639,0,0.008335,0
367,97453,img/97453.png,0,potato is my favorite color of the alphabet,0.138498,0,0.002182,0
368,98235,img/98235.png,0,fashion forward !,0.013069,0,0.017090,0
369,98543,img/98543.png,1,upper downer,0.716129,1,0.957242,1


In [ ]:
base_true = dev_vil_base.loc[dev_vil_base['truth'] == dev_vil_base['baseline_label']]
base_true[base_true['truth'] != base_true['vilio_label']]

,id,img,truth,text,vilio_proba,vilio_label,baseline_proba,baseline_label
0,1268,img/01268.png,0,father and daughter bonding time is the best l...,0.659967,1,0.000417,0
7,2145,img/02145.png,0,"and then i asked mom, what's for dinner?",0.951114,1,0.002921,0
9,2478,img/02478.png,1,must..resist...urge to.. kill whitey.,0.091324,0,0.950066,1
24,4621,img/04621.png,0,you are what you eat meanwhile in africa,0.707168,1,0.009650,0
26,5279,img/05279.png,1,"knowing white people , that's probably the bab...",0.412886,0,0.967358,1
...,...,...,...,...,...,...,...,...
356,95613,img/95613.png,1,what a weird way to spell fat and single,0.219216,0,0.990125,1
362,96312,img/96312.png,0,when you support your little one every step of...,0.701753,1,0.000296,0
363,96472,img/96472.png,1,white power!,0.105513,0,0.976899,1
365,97305,img/97305.png,1,when your wife just died giving birth to your ...,0.434577,0,0.985598,1


In [ ]:
vbens = pd.read_csv('visualbert_ensemble_devseen.csv', names=['id', 'proba', 'label']).sort_values(by='id')
vbens

,id,proba,label
494,1268,0.000230,0
159,1456,0.964295,1
332,1726,0.995897,1
220,1742,0.000248,0
217,1765,0.999624,1
...,...,...,...
457,98235,0.000237,0
412,98543,0.999641,1
271,98547,0.000967,0
164,98701,0.000257,0


In [ ]:
rename_columns_1 = {'label_x': 'truth', 'label_y': 'vilio_label', 'label': 'vb_label', 'proba_x': 'vilio_proba', 'proba_y': 'vb_proba'}
rename_columns_2 = {'proba': 'base_proba', 'label': 'base_label'}
dev_vil_vb = pd.merge(left=dev_vil, right=vbens, how='inner', left_on='id', right_on='id').rename(columns=rename_columns_1)
dev_vil_vb_base = pd.merge(left=dev_vil_vb, right=baseline, how='inner', left_on='id', right_on='id').rename(columns=rename_columns_2)
dev_vil_vb_base

,id,img,truth,text,vilio_proba,vilio_label,vb_proba,vb_label,base_proba,base_label
0,1268,img/01268.png,0,father and daughter bonding time is the best l...,0.659967,1,0.000230,0,0.000417,0
1,1456,img/01456.png,1,they see them rollin..... they hating..,0.768284,1,0.964295,1,0.100073,0
2,1726,img/01726.png,1,a real man loads the dishwasher every night,0.909551,1,0.995897,1,0.837024,1
3,1796,img/01796.png,1,life hack #23 how to get stoned with no weed,0.160565,0,0.067211,0,0.269773,0
4,1925,img/01925.png,1,i am not racist i just don't like brown sugar,0.183132,0,0.002556,0,0.000957,0
...,...,...,...,...,...,...,...,...,...,...
367,97453,img/97453.png,0,potato is my favorite color of the alphabet,0.138498,0,0.000370,0,0.002182,0
368,98235,img/98235.png,0,fashion forward !,0.013069,0,0.000237,0,0.017090,0
369,98543,img/98543.png,1,upper downer,0.716129,1,0.999641,1,0.957242,1
370,98701,img/98701.png,1,oh no i can't steal anymore,0.093028,0,0.000257,0,0.000820,0


In [ ]:
vil_true = dev_vil_vb_base[dev_vil_vb['truth'] == dev_vil_vb['vilio_label']]
vil_true_base_false = vil_true[vil_true['vilio_label'] != vil_true['base_label']]
vil_true_base_false[vil_true_base_false['vilio_label'] != vil_true_base_false['vb_label']]

,id,img,truth,text,vilio_proba,vilio_label,vb_proba,vb_label,base_proba,base_label
16,3519,img/03519.png,1,i'm going home i'm russian,0.920746,1,0.024671,0,0.156326,0
17,3524,img/03524.png,1,terrorist can come to this country way to easy,0.578985,1,0.000926,0,0.035441,0
20,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,0.766871,1,0.000550,0,0.001387,0
36,7135,img/07135.png,1,this is the greatest news manufacturing in 20t...,0.889013,1,0.002013,0,0.000709,0
41,7653,img/07653.png,1,in the last days there will be god hating scof...,0.743632,1,0.001050,0,0.107709,0
50,10785,img/10785.png,1,"time to put the ""panic"" back in hispanic",0.664788,1,0.000331,0,0.001090,0
59,13809,img/13809.png,1,doing o's with the smoke,0.949762,1,0.000222,0,0.001561,0
71,17682,img/17682.png,1,give them what they deserve kill them all,0.885749,1,0.000516,0,0.141564,0
86,19530,img/19530.png,1,they shot the wrong gorilla,0.504579,1,0.000329,0,0.000702,0
95,23810,img/23810.png,1,these aren't people,0.609831,1,0.000254,0,0.000751,0


In [ ]:
vil_false_base_vb_false = dev_vil_vb_base[dev_vil_vb_base['truth'] != dev_vil_vb_base['base_label']]
vil_false_base_vb_false = vil_false_base_vb_false[vil_false_base_vb_false['truth'] != vil_false_base_vb_false['vb_label']]
vil_false_base_vb_false = vil_false_base_vb_false[vil_false_base_vb_false['truth'] != vil_false_base_vb_false['vilio_label']]
vil_false_base_vb_false

,id,img,truth,text,vilio_proba,vilio_label,vb_proba,vb_label,base_proba,base_label
3,1796,img/01796.png,1,life hack #23 how to get stoned with no weed,0.160565,0,0.067211,0,0.269773,0
4,1925,img/01925.png,1,i am not racist i just don't like brown sugar,0.183132,0,0.002556,0,0.000957,0
11,2568,img/02568.png,1,whatsthe difference between heroin and i? cops...,0.267646,0,0.001022,0,0.012639,0
30,6123,img/06123.png,1,a group of men,0.169105,0,0.000231,0,0.020122,0
31,6273,img/06273.png,1,the jihad squad,0.222964,0,0.010732,0,0.003151,0
...,...,...,...,...,...,...,...,...,...,...
334,91756,img/91756.png,1,abnormal vs normal,0.291494,0,0.002291,0,0.002252,0
344,93148,img/93148.png,0,the proper way to pop a blackhead,0.918646,1,0.993338,1,0.727309,1
350,94560,img/94560.png,1,and for my next class project!,0.121906,0,0.002251,0,0.000987,0
358,95830,img/95830.png,1,face of terrorism in us,0.092836,0,0.000281,0,0.087466,0


In [ ]:
devseen[devseen['label'] == 0]

,id,img,label,text
494,1268,img/01268.png,0,father and daughter bonding time is the best l...
220,1742,img/01742.png,0,in just one hour from now i'll only have 4 hou...
206,1967,img/01967.png,0,when you decide to stop racism by burning its ...
290,2145,img/02145.png,0,"and then i asked mom, what's for dinner?"
365,2364,img/02364.png,0,"a jewish shop owner, wears a kippah or yarmulk..."
...,...,...,...,...
499,97320,img/97320.png,0,"""when your dishwasher is broken so you take it..."
90,97453,img/97453.png,0,potato is my favorite color of the alphabet
457,98235,img/98235.png,0,fashion forward !
271,98547,img/98547.png,0,waves mexican flag doesn't want to live in mexico


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58e10117-68e0-4aa0-b850-9c3a00ef4474' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>